### Import modules

In [1]:
import pandas as pd
import folium

### File description
* file pyramids.csv contains 62 Egyptian pyramids;
* every pyramid has a Modern Name and some have an Ancient Name as it was translated from ancient egyptian hieroglyphs;
* every pyramid has its coordinates: Latitude and Longitude in decimal form;
* there is an owner of pyramid. Symbol '(?)' marks a supposed owner;
* every pyramid is marked with its Dynasty and location (Site);
* there is missing data (Slope and Volume mostly);
* base (Base1 (m) and Base2 (m)) and Height (m) of pyramid are in meters and represent the original size (that the pyramid was or was supposed to be);
* some pyramids are marked with a Lepsius number which is a roman number from I to LXVII plus the pyramid number LG100 (pyramid of Khentkaus I);
* most pyramids have a Type of construction (Step, True, Smooth-faced, Smooth-sided) and most used Material (Limestone, Mudbrick, Granite, Sandstone or a combination);
* there is also a column Comment which contains some descriptive information.

In [7]:
prmds = pd.read_csv('../pyramids.csv') 
prmds

,Pharaoh,Ancient name,Modern name,Dynasty,Site,Base1 (m),Base2 (m),Height (m),Slope (dec degr),Volume (cu.m),Latitude,Longitude,Type,Lepsius,Material,Comment
0,Djoser,Hiemlender,Step Pyramid of Djoser,3,Saqqara,121.00,109.00,60.00,NaN,330.4,29.871215,31.216644,Step,XXXII,Mudbrick,NaN
1,Sekhemkhet,-,Buried Pyramid,3,Saqqara,120.00,120.00,70.00,NaN,33.6,29.866000,31.213000,Step,NaN,Mudbrick,(unfinished)
2,Khaba (?),-,Layer Pyramid,3,Zawyet el'Aryan,84.00,84.00,40.00,NaN,47.04,29.932820,31.161262,Step,XIV,Mudbrick,(unfinished)
3,Huni,-,Pyramid Lepsius I,3,Abu Rawash,215.00,215.00,145.00,NaN,NaN,30.040833,31.094444,Step,I,Mudbrick,totally ruined
4,Huni (?),-,Pyramid of Athribis,3,Athribis,20.00,20.00,16.00,50.0,NaN,30.470556,31.188056,Step,NaN,Mudbrick,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,Khendjer,-,Pyramid of Khendjer,13,South Saqqara,52.50,52.50,37.35,55.0,44.096,29.832222,31.223889,True,XLIV,Mudbrick,SIP
58,Weserkhau (?),-,Southern South Saqqara pyramid,13,South Saqqara,78.75,78.75,NaN,NaN,NaN,29.830556,31.222222,True,XLVI,Mudbrick,SIP
59,Neferhotep I,-,Tomb S9,13,Abydos,45.00,60.00,NaN,NaN,NaN,26.171389,31.925000,Mastaba,NaN,Mudbrick,SIP
60,Sobekhotep IV,-,Tomb S10,13,Abydos,40.00,30.00,NaN,NaN,NaN,26.171111,31.924167,Mastaba,NaN,Mudbrick,SIP


### Get some statistics
* Maximum height of pyramid is 145 meters (ruined Great Pyramid of Giza (also known as the Pyramid of Khufu or the Pyramid of Cheops));
* Minimum height of pyramid is 6.8 m (Pyramid of Seila).

In [3]:
prmds.describe()

,Dynasty,Base1 (m),Base2 (m),Height (m),Slope (dec degr),Latitude,Longitude
count,62.000000,62.000000,62.000000,53.000000,43.000000,62.000000,62.000000
mean,6.403226,78.977581,78.463065,47.817547,57.121628,29.226241,31.319200
std,3.889708,54.685717,54.199409,34.198331,11.411212,1.497457,0.446828
min,3.000000,18.000000,18.000000,6.800000,33.000000,24.085556,30.828611
25%,4.000000,41.100000,44.750000,18.000000,51.920000,29.621458,31.136257
50%,5.000000,72.650000,72.650000,47.000000,53.130000,29.850834,31.212611
75%,8.000000,105.000000,105.000000,60.000000,56.785000,29.924281,31.221598
max,18.000000,230.300000,230.300000,146.600000,82.000000,30.470556,32.885278


### Arrange pyramids into periods
* Find mean latitude and longitude of dataset;
* Create function to color different kingdoms;
* Provide a legend.

In [8]:
lat_mean = prmds['Latitude'].mean()
lon_mean = prmds['Longitude'].mean()

periods = {'EDP': (1,2), 'Old': (3,4,5,6,7), 'FIP': (8,9,10), 'Middle': (11,12,13,14), 'SIP': (15,16,17), 'New': (18,)}
colors_dyn = {'EDP':'violet', 'Old': 'green', 'FIP': 'blue', 'Middle': 'orange', 'SIP': 'yellow', 'New': 'red'}

def col_dyn(dyn):
    for k, v in periods.items():
        if dyn in v:
            c = colors_dyn[k]
            break
        else:
            c = 'black'
    return c

legend_kingdom = '''
     <div style= "position: fixed; 
     bottom: 45px; left: 15px; width: 150px; height: 150px;white-space: pre-line; 
     border:1px solid grey; z-index:9999; font-size:10px; background-color:white;
     ">&nbsp; <b>Period</b>
    &nbsp; Early Dyn. Period &emsp;<i class="fa fa-caret-up fa-2x"
                  style="color:violet"></i>
     &nbsp; Old Kingdom &emsp;<i class="fa fa-caret-up fa-2x"
                  style="color:green"></i>
     &nbsp; FIP &emsp;<i class="fa fa-caret-up fa-2x"
                  style="color:lightblue"></i>
     &nbsp; Middle Kingdom &emsp;<i class="fa fa-caret-up fa-2x"
                  style="color:orange"></i>
    &nbsp; SIP &emsp;<i class="fa fa-caret-up fa-2x"
                  style="color:yellow"></i>
    &nbsp; New Kingdom &emsp;<i class="fa fa-caret-up fa-2x"
                  style="color:red"></i>
      </div>
     '''

### Map with all known pyramids
* Result: Map with pyramids on it! docs folder;
* glyphicon with open eye shows if Pharaoh is certain.

In [11]:
m = folium.Map(location = [lat_mean, lon_mean], #center of a map
               zoom_start=6, min_zoom = 5, width = 600, control_scale = True)

for i in range(0,len(prmds)):
    lat = prmds.iloc[i]['Latitude']
    lon = prmds.iloc[i]['Longitude']
    folium.Marker([lat, lon], popup=prmds.iloc[i]['Modern name'], # show a modern name of pyramid in popup
                  icon=folium.Icon(color=col_dyn(prmds.iloc[i]['Dynasty']),
                                   icon='eye-close' if str(prmds.iloc[i]['Pharaoh']).find("?")==-1 else 'eye-open') # glyphicon with open eye if Pharaoh is certain
                 ).add_to(m)
m.get_root().html.add_child(folium.Element(legend_kingdom))
display(m)

m.save(outfile= "../docs/map-pyramids.html")

### Lepsius list and map
* Drop rows where Lepsius column contains NaN values;
* Create Lepsius map: zoom in to get the pyramids and the Pharaoh's name.

In [12]:
prmds_Le = prmds.dropna(subset=['Lepsius'])
m2 = folium.Map(location = [prmds_Le['Latitude'].mean(), prmds_Le['Longitude'].mean()],
               zoom_start=9, min_zoom = 6, width = 600)

for i in range(0,len(prmds_Le)):
    lat = prmds_Le.iloc[i]['Latitude']
    lon = prmds_Le.iloc[i]['Longitude']
    temp = prmds_Le.iloc[i]['Lepsius']
    folium.CircleMarker([lat, lon],  radius=8,  color='red', fill=True, fill_color='red').add_to(m2)
    folium.Marker([lat, lon], 
                  popup=prmds_Le.iloc[i]['Pharaoh'], 
                         icon=folium.DivIcon(html=f'''<div style="font-family: arial; color: 'black'">{"{}".format(temp)}</div>''')
                
        ).add_to(m2)
display(m2)

m2.save(outfile= "../docs/map-Lepsius.html")